# TraceBERT
### Objective
In this notebook, we aim to train a BERT-style model using standard MLM on an _abstracted_ Java trace dataset consisting of two datasets generated from Commonsio and Nettyio.

To achieve this and evaluate our performance, we apply the following methodology:
1. Train a tokenizer specifically designed for our data, which occurs in the form
```
performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]
```

We want each token to be either the method name e.g. _performExpensiveLogSetup_ or an event in the trace _[EXIT]_. This allows us to represent long traces without using a fancy transformer.

2. We define a model based on BERT, but somewhat smaller.

| Hyperparameter               | Value |
|------------------------------|-------|
| vocab_size                   | 9582  |
| max_position_embeddings      | 768   |
| intermediate_size            | 2048  |
| hidden_size                  | 512   |
| num_attention_heads          | 8     |
| num_hidden_layers            | 6     |
| type_vocab_size              | 5     |
| hidden_dropout_prob          | 0.1   |
| attention_probs_dropout_prob | 0.1   |

3. We load our dataset (Commonsio traces) and split by train/dev/test (dev overlaps with train, dev does not).

4. We tokenize our dataset, and proceed to inspect it to see some of the most frequent function names across the train/dev/test split.

5. We define a trainer class to pretrain a BERT model from scratch. During training, we evaluate our performance on the dev set and plot our loss curve for a heldout slice of the training dataset.

6. We begin our model interpretation by loading a new dataset consisting of Nettyio traces. This serves as a true evaluation set, as presumably Nettyio traces are more distant from Commonsio traces. We extract all examples of the `compare` method. We do the same for our Commonsio training dataset, and slice the datasets such that their lengths are equal (i.e. if there are 1500 examples from Commonsio, and 200 from Nettyio, we select the first 200 from Commonsio).

7. We proceed to define a function to extract the embedding of the [CLS] token from the trained model. We then extract the embeddings for ten thousand examples from Commonsio, the Commonsio and Nettyio `compare` examples, and extract embeddings for another Commonsio function, `checkPositionIndexes`.

8. We then construct various FAISS (vector search) indexes on the first 10k examples from Commonsio (compute reasons). These indexes allow us to search in the vector space for similar embeddings based on Inner Product and Euclidean distances. We show that embeddings are able to find similar functions in the space. We also calculate cosine similarities, and perform a t-test on a sample of distances of embeddings of the same function, and another sample of embeddings from a different function.

9. We then proceed to perform a similar analysis using tSNE projection/clustering, showing that functions of the same name cluster, and different functions cluster elsewhere. We also cluster ten thousand traces from Commonsio.

10. As a toy, we demonstrate that TraceBERT is able to recover missing calls and function names.

Re-calc embeddings without function

### Generic imports

In [ ]:
# Force tokenizers to run in parallel
%env TOKENIZERS_PARALLELISM=true
%env HF_DATASETS_CACHE=hf_cache
%env NCCL_DEBUG=INFO
%env CUDA_VISIBLE_DEVICES=2,4,5
import wandb
import torch
import numpy as np
import faiss
from collections import Counter
import matplotlib.pyplot as plt

# Check if CUDA is available (Docker attached correctly)
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
print(torch.cuda.device_count())

In [ ]:
wandb.login() # log in weights & biases

### Trace Tokenizer

In [ ]:
from tokenizers import Tokenizer, Regex
from tokenizers.models import WordLevel
from tokenizers.processors import BertProcessing
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Split
from tokenizers.normalizers import Strip

#### Tokenizer set up

In [ ]:
exp = Regex("(^((\w)+(?=\s)))|((\[ENTRY\]\ (\w|\||\.)+)\s)|((\[CALL\]\ (\w|\||\.|\s)+)(?=\ \[))+|(\[EXIT\])")

trace_tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
trace_tokenizer.add_special_tokens(["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])#, "[ENTRY]", "[EXIT]", "[CALL]")
trace_tokenizer.normalizer = Strip()
trace_tokenizer.pre_tokenizer = Split(pattern=exp, behavior="removed",invert=True)
trace_tokenizer.post_processor = BertProcessing(sep=("[SEP]", 0),cls=("[CLS]", 1))
VOCAB_SIZE = 10000
trainer = WordLevelTrainer(
    vocab_size=VOCAB_SIZE, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"]
)

In [ ]:
# Sanity check on the pre_tokenizer
trace_tokenizer.pre_tokenizer.pre_tokenize_str(
        (
        "[ENTRY] void "
        "[CALL] java.io.PrintStream println java.lang.String void "
        "[CALL] java.lang.Math pow double|double double "
        "[CALL] java.lang.Math sqrt double double "
        "[CALL] java.io.PrintStream println java.lang.String void "
        "[EXIT]")
    )


#### Train tokenizer

In [ ]:
#t = Tokenizer.from_file("trace_tokenizer_google.json")
import re

In [ ]:
dict(filter(lambda elem: re.search('^(?!\[ENTRY\]).*',elem[0]),trace_tokenizer.get_vocab().items()))

In [ ]:
{k: v for k, v in sorted(t.get_vocab().items(), reverse=True, key=lambda item: item[1])}

In [ ]:
trace_tokenizer.get_vocab_size()

In [ ]:
# Sanity check on the pre_tokenizer
t.pre_tokenizer.pre_tokenize_str(
        (
        "[ENTRY] void "
        "[CALL] java.io.PrintStream println java.lang.String void "
        "[CALL] java.lang.Math pow double|double double "
        "[CALL] java.lang.Math sqrt double double "
        "[CALL] java.io.PrintStream println java.lang.String void "
        "[EXIT]")
    )

cut -d ' ' -f 2- google_uniq.txt > google_uniq_vocab.txt

sort -u netty.txt | uniq -u | shuf > netty_uniq.txt

In [ ]:
files = ["../projsplit/netty_uniq_vocab1.txt", "../projsplit/google_uniq_vocab1.txt"] # Define files to train on
trace_tokenizer.train(files, trainer)
trace_tokenizer.save("trace_tokenizer_no_method_uniq.json") # Save tokenizer

In [ ]:
trace_tokenizer.get_vocab()

# TraceBERT Model

### Load pretrained tokenizer fast

HuggingFace nicely provided a high performance Rust implementation, let's use it.

In [ ]:
from transformers import PreTrainedTokenizerFast, BertTokenizerFast
fast_tokenizer = BertTokenizerFast(
                                        tokenizer_file="trace_tokenizer_no_method_uniq.json",
                                        return_special_tokens_mask=True,
                                        return_token_type_ids=True
                                    )

### Define TraceBERT config

There *is* a reason for defining the config so early in the notebook: when we encode our dataset we want to perform truncation/padding based on our model's size, so it is easier to know this in advance.

In [ ]:
from transformers import BertConfig

In [ ]:
scale_factor = 1 

tracebert_config = BertConfig(
    # VOCAB_SIZE plus some extra for special tokens
    vocab_size=(fast_tokenizer.vocab_size + 3),
    max_position_embeddings=int(768*scale_factor),
    intermediate_size=int(2048*scale_factor),
    hidden_size=int(512*scale_factor),
    num_attention_heads=8,
    num_hidden_layers=6,
    type_vocab_size=5,
    hidden_dropout_prob=0.1,
    attention_probs_dropout_prob=0.1,
    output_hidden_states=True,
    return_dict=True
)

#### Sanity check tokenizer

In [ ]:
fast_tokenizer.decode(fast_tokenizer.encode("[EXIT]"))

In [ ]:
fast_tokenizer.vocab_size

## Load datasets

In [ ]:
from datasets import load_dataset
import datasets

#### Train

In [ ]:
train_dataset = load_dataset(
                        'text',
                        data_files='../projsplit/both_r.txt',
                        split='train[:95%]' # Load first 5% of dataset
                    )

In [ ]:
# Sanity check
train_dataset

#### Dev

In [ ]:
google_dev_dataset = load_dataset(
                        'text',
                        data_files='../projsplit/google.txt',
                        split='train[:1500]' # WARNING: we are loading the first 1.5k rows for dev
                    )

In [ ]:
# Sanity check
google_dev_dataset

#### Test

In [ ]:
test_dataset = load_dataset(
                        'text',
                        data_files='../projsplit/both_r.txt',
                        split='train[95%:100%]' # Load 3k rows for test
                    )

In [ ]:
# Sanity check
test_dataset

## Tokenize/encode datasets

In [ ]:
# Define tokenization function
# Max length: length of model
# Truncation: true (defaults to max example length)
# Padding: true (until truncation length)
def tokenize_trace(examples):
    return {
        **fast_tokenizer(
            examples['method_body'],
            max_length = tracebert_config.hidden_size,
            truncation=True,
            padding=True
        )
    }

def extract_trace(examples):
        return { 'method_name': [ t[:t.index(' ')] for t in examples['text'] ],
        'method_body': [ t[t.index(' ')+1:] for t in examples['text'] ] }

#### Tokenize training dataset

In [ ]:
train_dataset_enc = (train_dataset
                            .map(
                                            extract_trace,
                                            batched=True
                                        )
                            .remove_columns('text')
                            .map(
                                            tokenize_trace,
                                            batched=True,
                                num_proc=32
                                        )
                           )

In [ ]:
# Sanity check
train_dataset_enc[0]

In [ ]:
# Save tokenized version
train_dataset_enc.save_to_disk('datasets/both_uniq_no_name')

#### Tokenize dev dataset

In [ ]:
google_dev_dataset_enc = google_dev_dataset.map(
                                            extract_trace,
                                            batched=True
                                        ).remove_columns('text')

In [ ]:
# Sanity check
google_dev_dataset_enc.to_csv('google_dev_dataset_test')#, num_proc=64)

In [ ]:
# Save tokenized version
google_dev_dataset_enc.save_to_disk('datasets/google_dev_dataset_enc')

#### Tokenize test dataset

In [ ]:
test_dataset_enc = (test_dataset.map(
                                            extract_trace,
                                            batched=True
                                        ).remove_columns('text')
                            .map(
                                            tokenize_trace,
                                            batched=True,
                                num_proc=32
                                        )
                           )

In [ ]:
# Sanity check
test_dataset_enc

In [ ]:
# Save tokenized version
test_dataset_enc.save_to_disk('datasets/test_dataset_enc_uniq_no_name')

### Load tokenized datasets (if already saved)

In [ ]:
train_dataset_enc = datasets.load_from_disk('datasets/both_uniq_no_name')
#google_dev_dataset_enc = datasets.load_from_disk('datasets/google_dev_dataset_enc')
test_dataset_enc = datasets.load_from_disk('datasets/test_dataset_enc_uniq_no_name')


## Inspect datasets

In [ ]:
train_func_frequencies = Counter(google_train_dataset_enc['method_name']).most_common()

In [ ]:
train_func_frequencies

In [ ]:
dev_func_frequencies = Counter(google_dev_dataset_enc['func_name']).most_common()

In [ ]:
test_func_frequencies = Counter(google_test_dataset_enc['func_name']).most_common()

In [ ]:
def plot_freq(counter):
    counts = [ (index, value[1]) for index, value in enumerate(counter) ]
    plt.scatter(*zip(*counts))
    plt.show()

In [ ]:
plot_freq(train_func_frequencies[:100])
print(train_func_frequencies[:15])

## Define Trainer for experiment

In [ ]:
from transformers import TrainingArguments, DataCollatorForWholeWordMask, Trainer, BertForMaskedLM

We define our model from the BertForMaskedLM class

In [ ]:
tracebert_model = BertForMaskedLM(tracebert_config)

In [ ]:
tracebert_model.save_model()

In [15]:
# performs masking
data_collator = DataCollatorForWholeWordMask(
                    tokenizer=fast_tokenizer,
                    mlm=True,
                    mlm_probability=0.15
                )

training_args = TrainingArguments(
    "tracebert_013", # Model name/directory
    run_name='tracebert_20220214_1', # experiment name
    # group_by_length=True, # minimize padding
    # overwrite the content of the output directory
    # continue training if output_dir points to a checkpoint directory.
    overwrite_output_dir=True,
    report_to = 'wandb',
    num_train_epochs=300, # defaults to 3
    prediction_loss_only=False, # only return loss during eval
    save_steps=1000, # save checkpoint every 5k steps
    save_total_limit=3, # save only last 3 checkpoints (we're low on space ☹️)
    #eval_accumulation_steps=1000, # number of steps to accumulate before moving to CPU, enable if running out of CUDA RAM
    logging_steps=100, # num steps before train loss is logged
    eval_steps=100, # num steps before eval loss is calculated
    # max_steps=1000, # max number of steps to train
    save_strategy='steps',
    evaluation_strategy='steps',
    per_device_train_batch_size=44, # increase batch size
    per_device_eval_batch_size=16, # increase batch size
)


trainer = Trainer(
    model=tracebert_model,
    tokenizer=fast_tokenizer,
    data_collator=data_collator,
    args=training_args,
    train_dataset=train_dataset_enc, # DEFINE DATASETS !!!
    eval_dataset=test_dataset_enc,
    #bf16=True,
    #fp16=False,
    #fp16_opt_level='01', # [‘O0’, ‘O1’, ‘O2’, and ‘O3’]
    #half_precision_backend='auto', # apex or amp
    #bf16_full_eval=False,
    #fp16_full_eval=False,
    #sharded_ddp=False,
    #deepspeed=None, # json file or dict or json
    #compute_metrics=compute_metric, # if custom metric function available
)

PyTorch: setting up devices


In [17]:
train_dataset_enc

Dataset({
    features: ['method_name', 'method_body', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 57103
})

#### Train model

# Currently does 52 steps/min
***** Running training *****
  Num examples = 3376066
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 79128

In [ ]:
torch.cuda.empty_cache()

In [ ]:
%%wandb
trainer.train(resume_from_checkpoint=True)

Loading model from tracebert_013/checkpoint-12000).
The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: method_name, method_body.
***** Running training *****
  Num examples = 57103
  Num Epochs = 300
  Instantaneous batch size per device = 44
  Total train batch size (w. parallel, distributed & accumulation) = 132
  Gradient Accumulation steps = 1
  Total optimization steps = 129900
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 27
  Continuing training from global step 12000
  Will skip the first 27 epochs then the first 309 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/309 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/opt/conda/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
12100,0.812600,0.740149
12200,0.797100,0.660017
12300,0.781300,0.663482
12400,0.729400,0.653488
12500,0.684100,0.628976
12600,0.664300,0.591635
12700,0.649800,0.576799
12800,0.631900,0.556889
12900,0.602100,0.564144
13000,0.592300,0.544180


The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: method_name, method_body.
***** Running Evaluation *****
  Num examples = 3005
  Batch size = 48
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: method_name, method_body.
***** Running Evaluation *****
  Num examples = 3005
  Batch size = 48
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: method_name, method_body.
***** Running Evaluation *****
  Num examples = 3005
  Batch size = 48
The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: method_name, method_body.
***** Running Evaluation *****
  Num examples = 3005
  Batch size = 48
The following columns in the evaluation set  don't have a corresponding argument

In [ ]:
wandb.finish()

In [18]:
# Save model manually if needed
trainer.save_model()

Saving model checkpoint to tracebert_013
Configuration saved in tracebert_013/config.json
Model weights saved in tracebert_013/pytorch_model.bin
tokenizer config file saved in tracebert_013/tokenizer_config.json
Special tokens file saved in tracebert_013/special_tokens_map.json


In [ ]:
trainer.evaluate()

# Interpret model

### Retrieve examples of specific function name

In [ ]:
def exact_str_match(x,y):
    return x==y
def str_contains(x,y):
    return x in y

# pass comparator which (x,y) where x is needle and y is haystack
def retrieve_func_examples(dataset, function_name, comparator=exact_str_match):
    return dataset.filter(lambda trace: comparator(function_name, trace['method_name']))


In [ ]:
func_compare_google = retrieve_func_examples(
                            google_train_dataset_enc,
                            'compare',
                            comparator=exact_str_match
                        )

In [ ]:
func_compare_google

#### Load Netty dataset

In [ ]:
netty_dataset_enc = (
                        load_dataset(
                            'text',
                            data_files='../projsplit/netty.txt',
                            split='train[:10000]')
                        .map(
                        # WARNING: using a tokenizer NOT trained on this dataset
                        # could result in poor performance due to UNKs
                            tokenize_trace,
                            batched=True
                        )
                    )

### Retrieve all examples of 'compare' function

In [ ]:
func_compare_netty = retrieve_func_examples(
                            netty_dataset_enc,
                            'compare',
                            comparator=exact_str_match
                        )

#### Find the minimum number of examples of func in both datasets (class balance)

In [ ]:
minimum_funcs = min(len(func_compare_netty), len(func_compare_google))
minimum_funcs

In [ ]:
func_compare_netty = func_compare_netty.select(range(0, minimum_funcs))
func_compare_google = func_compare_google.select(range(0, minimum_funcs))

#### Pick another function from Commonsio for comparisons

In [ ]:
func_CPI_google = retrieve_func_examples(
                            google_train_dataset_enc,
                            'checkPositionIndexes',
                            comparator=exact_str_match
                        )

Now we have two datasets with an equal amount of examples for a particular method, we now load these examples into the model and extract the embeddings

# (Re)load pretrained model for embeddings

If the model is not loaded yet, select a pretrained model to generate embeddings

In [ ]:
# tracebert_model = BertForMaskedLM.from_pretrained('test_trainer_bert_pre_2')
# or 
# or BertModel.from_pretrained('path_to_model_dir', add_pooling_layer=True)
tracebert_model

We must ensure/assert that the model class will return the hidden states

In [ ]:
tracebert_model.config.output_hidden_states = True
tracebert_model.config.return_dict = True
assert tracebert_model.config.output_hidden_states == True

Define a function to load examples and extract hidden states from model as numpy arrays

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def calculate_embeddings(examples, method='hidden_state'):
    with torch.no_grad(): # no autograd if we don't train
        if method == 'hidden_state':
            # get the hidden state of the [CLS] token
            # we first grab the hidden states from the model,
            # then take the last layer, [-1], and convert it to a 3D
            # numpy array. N is number of examples in batch.
            # L is number of tokens (longest example fixes L)
            # D is number of hidden states
            # the shape is (N, L, D)
            # since [CLS] is always first, if we slice
            # (N, L[:1], D) we will get the hidden dim of the [CLS] token.
            # since L is now always 1, it is useless, and we can squeeze it out
            # now the shape is (N, D) where each example has 512 vectors
            #print(f"input_ids={torch.tensor(examples['input_ids']).shape}, attention_mask={torch.tensor(examples['attention_mask']).shape}, token_type_ids={torch.tensor(examples['token_type_ids']).shape}")
            b = tracebert_model(
                        input_ids=pad_sequence([torch.tensor(x) for x in examples['input_ids']], batch_first=True),
                        attention_mask=pad_sequence([torch.tensor(x) for x in examples['attention_mask']], batch_first=True),
                        token_type_ids=pad_sequence([torch.tensor(x) for x in examples['token_type_ids']], batch_first=True)
                    ).hidden_states[-1].numpy()[:,:1,:].squeeze()
            return { 'embedding': b }
        elif method == 'pooler_output':
            return tracebert_model(examples).pooler_output.numpy() # shape [n, 512]

In [ ]:
func_compare_google

In [ ]:
google_compare_emb = func_compare_google.map(calculate_embeddings, batched=True)

In [ ]:
netty_compare_emb = func_compare_netty.map(calculate_embeddings, batched=True)

In [ ]:
func_CPI_google_emb = func_CPI_google.select(range(0,250)).map(calculate_embeddings, batched=True)

Sanity check that shapes match

In [ ]:
assert np.array(google_compare_emb['embedding']).shape[0] == np.array(netty_compare_emb['embedding']).shape[0]
assert np.array(google_compare_emb['embedding']).shape[1] == np.array(netty_compare_emb['embedding']).shape[1]
assert np.array(google_compare_emb['embedding']).shape[0] == minimum_funcs
assert np.array(google_compare_emb['embedding']).shape[1] == tracebert_model.config.hidden_size

## Add embeddings to dataset and search

We start by selecting a more workable subset of the Google dataset

In [ ]:
ten_k_google = (
                google_train_dataset_enc
                .select(range(0,10000))
                .map(
                      calculate_embeddings,
                      batched=True
                    )
                )

We add a FAISS index to make this dataset searchable

TODO: A custom index can/should be used to improve performance. There are many, many indexes to choose from and have complexity and space considerations, and exhaustion considerations. We can probably handle much, much more traces if we figure this out. Upfront compute is probably higher as well.

In [ ]:
# this is the DEFAULT config, seems to be flat index with IP metric
# note there is a problem with this index: the vectors are supposed to be
# normalized for inner product if we want to find cosine similarity
ten_k_google.add_faiss_index(
    column='embedding',
    index_name='index_flat_ip',
    #device=0, # GPU index
    # passed to faiss.index_factory(), defaults to IndexFlatIP
    string_factory='Flat',
    metric_type=faiss.METRIC_INNER_PRODUCT, # or faiss.METRIC_INNER_PRODUCT
    #custom_index= can specify a custom index object for more power
    #train_size: Optional[int] = None,
    faiss_verbose=True
)

In [ ]:
# let's add an index with L2 distance
ten_k_google.add_faiss_index(
    column='embedding',
    index_name='index_flat_l2',
    #device=0, # GPU index
    string_factory='Flat',
    metric_type=faiss.METRIC_L2,
    #custom_index= can specify a custom index object for more power
    #train_size: Optional[int] = None,
    faiss_verbose=True
)

Let's check if we can access our index directly...

In [ ]:
faiss_index = ten_k_google.get_index('index_flat_ip') # this should return a FAISS object
faiss_index

We save the index so that we don't have to recalculate next time

In [ ]:
ten_k_google.save_faiss_index('index_flat_ip', 'ten_k_google_ip.faiss')
ten_k_google.save_faiss_index('index_flat_l2', 'ten_k_google_l2.faiss')

As a sanity check, we search for an example in the dataset/index and evaluate the top ten results

In [ ]:
random_example_in = ten_k_google[np.random.randint(0, len(ten_k_google))]
random_example_in

In [ ]:
np.array(random_example_in['embedding']).shape

In [ ]:
scores, examples = ten_k_google.get_nearest_examples('index_flat_ip',
                                  np.array(random_example_in['embedding']).astype('float32'),
                                  k=10)

In [ ]:
random_example_in['text']

In [ ]:
# so why are traces with the exact same 'text' not the same distance?

In [ ]:
list(zip(scores, np.sqrt(scores), examples['text']))

We then load one example not in the smaller dataset

In [ ]:
random_example_out = google_train_dataset_enc[np.random.randint(10000, len(google_train_dataset_enc))]

In [ ]:
random_example_out['embedding'] = np.array(calculate_embeddings({'input_ids': [random_example_out['input_ids']],
                                                        'token_type_ids': [random_example_out['token_type_ids']],
                                                        'attention_mask': [random_example_out['attention_mask']] })['embedding'])

In [ ]:
# Sanity check on embedding dim
assert random_example_out['embedding'].shape == (512,)
random_example_out['embedding'].shape

We then search for the example outside of the dataset and evaluate the top 10 results

In [ ]:
# inner product
scores, results = ten_k_google.get_nearest_examples('index_flat_ip', np.array(random_example_out['embedding']).astype('float32'), k=10)


In [ ]:
random_example_out['func_name']

In [ ]:
list(zip(scores, examples['func_name']))

In [ ]:
# l2
ten_k_google.get_nearest_examples('ten_k_google_l2', np.array(random_example_out['embedding']).astype('float32'), k=3)


In [ ]:
# l2 search/distance
distance, index = ten_k_google.get_index('index_flat_l2').search(np.array(random_example_out['embedding']).astype('float32'), k=3)
print(f"Distance by FAISS: {np.sqrt(distance)}, index: {index}")

#### Cosine similarity index + search

Let's create a new index so we can calculate cosine similarity

In [ ]:
random_example_emb = np.array(random_example_out['embedding']).astype('float32')
#x = np.array([random_example_emb])#.astype(np.float32)
#q = np.array([random_example_emb])#.astype(np.float32)
faiss_l2_index = faiss.index_factory(512, "Flat", faiss.METRIC_L2)

For cosine similarity, we *must* normalize the index vectors and the search vectors

In [ ]:
ten_k_emb_norm = np.array(ten_k_google['embedding']).astype('float32')
random_example_emb_in = np.array([random_example_in['embedding']]).astype('float32')
faiss.normalize_L2(ten_k_emb_norm) # must normalize ALL vectors for this index
faiss_l2_index.train(ten_k_emb_norm)
# faiss.normalize_L2(random_example_emb)
faiss_l2_index.add(ten_k_emb_norm)

Sanity check:

In [ ]:
faiss_l2_index.is_trained

In [ ]:
faiss_l2_index.ntotal # should be one
#faiss_l2_index.train(training_vectors)
faiss_l2_index.add(random_example_emb)
distance, index = faiss_l2_index.search(random_example_emb, 1)
print(f"Distance by FAISS: {np.sqrt(distance)}, index: {index}")

In [ ]:
distance, index = faiss_inner_prod_index.search(random_example_emb, 1)
print(f"Distance by FAISS: {distance}, index: {index}")

In [ ]:
# calculate cosine similarity
from scipy import spatial, stats

In [ ]:
result = 1 - spatial.distance.cosine(ten_k_emb_norm[index[0]], random_example_emb)
print(f"Distance by scipy: {result}")

## t Test

In [ ]:
a = np.linalg.norm(np.array(google_compare_emb['embedding'][:50])-np.array(google_compare_emb['embedding'][50:100]))
b = np.linalg.norm(np.array(google_compare_emb['embedding'][:50])-np.array(func_CPI_google_emb['embedding'][:50]))
t, p = stats.ttest_ind(a, b, equal_var=False)
(t, p)

# tSNE: Are the embeddings really capturing something useful?

In [ ]:
from MulticoreTSNE import MulticoreTSNE as TSNE
#from tsnecuda import TSNE # pkg: tsnecuda
from matplotlib import cm
import matplotlib

In [ ]:
def draw_tsne(embeddings, colors):
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(18.5, 10.5)
    embeddings_tsne = TSNE(n_jobs=16).fit_transform(embeddings)
    vis_x_cb = embeddings_tsne[:, 0]
    vis_y_cb = embeddings_tsne[:, 1]
    fig = plt.figure(figsize=(36,24))
    ax = fig.add_subplot(111)
    ax.scatter(vis_x_cb, vis_y_cb, color=colors, marker='.')
    ax.set_aspect('equal', adjustable='datalim')
    plt.show()

In [ ]:
np.concatenate([np.array(netty_compare_emb['embedding']), np.array(google_compare_emb['embedding'])]).shape

In [ ]:
draw_tsne(
    np.concatenate([np.array(netty_compare_emb['embedding']), np.array(google_compare_emb['embedding'])]),
    cm.rainbow(np.concatenate((np.zeros(netty_compare_emb.shape[0]), np.ones(google_compare_emb.shape[0]))))
)

In [ ]:
draw_tsne(
    np.concatenate([np.concatenate([np.array(google_compare_emb['embedding']), np.array(func_CPI_google_emb['embedding'])])]),
    cm.rainbow(np.concatenate((np.zeros(google_compare_emb.shape[0]), np.ones(func_CPI_google_emb.shape[0]))))
)

In [ ]:
ten_k_emb = np.array(ten_k_google['embedding'])
draw_tsne(
    ten_k_emb,
    cm.rainbow(np.linspace(0, 1, ten_k_emb.shape[0])) # what /exactly/ do the colors represent?
)

# Does our model return something useful when asked to fill a mask?

In [ ]:
from transformers import pipeline
from pprint import pprint

unmasker = pipeline(
    "fill-mask",
    model=tracebert_model,
    tokenizer=fast_tokenizer
)

In [ ]:
pprint(
    unmasker((
        "setSuccessorInMultimap "
        "[ENTRY] void com.google.common.collect.LinkedHashMultimap$ValueEntry "
        f"{unmasker.tokenizer.mask_token} "
        "[EXIT]"
    )))

In [ ]:
pprint(
    unmasker((
        f"{unmasker.tokenizer.mask_token} "
        "[ENTRY] void com.google.common.collect.LinkedHashMultimap$ValueEntry "
        "[EXIT]"
    ))
)

# Misc leftovers

#### L2 distance index + search

In [ ]:
random_example_emb = random_example_out['embedding']
#x = np.array([random_example_emb])#.astype(np.float32)
#q = np.array([random_example_emb])#.astype(np.float32)
faiss_l2_index = faiss.index_factory(512, "Flat", faiss.METRIC_L2)

In [ ]:
faiss_l2_index.is_trained

In [ ]:
faiss_l2_index.ntotal # should be one
#faiss_l2_index.train(training_vectors)
faiss_l2_index.add(random_example_emb)
distance, index = faiss_l2_index.search(random_example_emb, 1)
print(f"Distance by FAISS: {np.sqrt(distance)}, index: {index}")

In [ ]:
ds_with_embeddings = encoded_dataset_test.map(lambda example:
                            {'embeddings': np.array([0])})
                             #bertmodel(**fast_tokenizer(example["text"], max_length = 512, truncation=True, padding=True, return_tensors="pt")).hidden_states[-1][0].numpy()},
                                      #  batched=True)

# ds_with_embeddings = encoded_dataset_train.map(lambda example:
#                             {'embeddings': 
#                              bertmodel(**{'attention_mask': torch.tensor(example['attention_mask']),
#                                          'input_ids': torch.tensor(example['input_ids']),
#                                          'token_type_ids': torch.tensor(example['token_type_ids'])})[0].numpy()},
#                                         batched=True)

#    e = np.save("google_ex", bertmodel(**fast_tokenizer(ten_k_google.select(range(0,222))["text"],
#                                  max_length = 512, truncation=True, padding=True, return_tensors="pt",
#                                  )).hidden_states[-1].detach().numpy())
# print(e.shape)

    
    bertmodel(**fast_tokenizer(ten_examples_google["text"],
                                   max_length = 512, truncation=True, padding=True, return_tensors="pt",
                                  )).hidden_states[-1].shape

 netty = np.save("netty_ex", bertmodel(**fast_tokenizer(ten_k_netty["text"],
  max_length = 512, truncation=True, padding=True, return_tensors="pt"
 )).hidden_states[-1].detach().numpy())
index = faiss.IndexFlatL2(512)   # build the index
print(index.is_trained)
index.search(hidden_states.detach().numpy(),k=1)
from torch.nn import functional as F
import torch
#subset = 
smaller_dataset = encoded_dataset_test.shard(num_shards=4000, index=0)
text = "The capital of France, " + fast_tokenizer.mask_token + ",contains the Eiffel Tower."
input = fast_tokenizer.encode_plus(text, return_tensors = "pt").to("cuda")
mask_index = torch.where(input["input_ids"][0] == fast_tokenizer.mask_token_id)
logits = model(**input)
logits = logits.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_word = torch.argmax(mask_word, dim=1)
print(fast_tokenizer.decode(top_word))

softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
for token in top_10:
   word = fast_tokenizer.decode([token])
   new_sentence = text.replace(fast_tokenizer.mask_token, word)
   print(new_sentence)
smaller_dataset
trainer_eval = trainer.evaluate(smaller_dataset, metric_key_prefix='no')

print(trainer_eval)
metrics = train_result.metrics
print(metrics)
#trainer.log_metrics("train", compute_metrics)
#trainer.save_metrics("train", compute_metrics)
#trainer.save_state()

print(metric.inputs_description)
model_predictions = model(smaller_dataset)
final_score = metric.compute(predictions=model_predictions, references=gold_references)

In [ ]:
test_input = fast_tokenizer(["performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [EXIT] [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]", "[EXIT]", ""], max_length = 512, truncation=True, padding=True, return_tensors="pt")
#fast_tokenizer.encode("performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [EXIT] [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]", max_length = 512, truncation=True, padding=True, return_tensors="pt")
hidden_states = bertmodel(**test_input).hidden_states

last_four_layers = [hidden_states[i] for i in (-1, -2, -3, -4)]
# cast layers to a tuple and concatenate over the last dimension
cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
print(cat_hidden_states.size())

# take the mean of the concatenated vector over the token dimension
cat_sentence_embedding = torch.mean(cat_hidden_states, dim=1).squeeze()
print(cat_sentence_embedding.size())
bertmodel(**fast_tokenizer("performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [EXIT] [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]", max_length = 512, truncation=True, padding=True, return_tensors="pt"))[1][0].numpy().shape
inputs = {
    "input_ids": encoded_dataset_test[0],
   #"attention_mask": batch[1]
}
input_sentence = torch.tensor(fast_tokenizer.encode("performExpensiveLogSetup [ENTRY] void [CALL] java.io.PrintStream println java.lang.String void [EXIT] [CALL] java.lang.Math pow double|double double [CALL] java.lang.Math sqrt double double [CALL] java.io.PrintStream println java.lang.String void [EXIT]", max_length = 512, truncation=True, padding=True)).unsqueeze(0)
output = bertmodel(input_sentence)
logits = output[0]
hidden_states = output[1]

In [ ]:
#assert both_embeddings.shape == (444, 512)

In [ ]:
#colors = cm.rainbow(np.linspace(0, 1, len(both_embeddings)))
colors = cm.rainbow(np.concatenate((np.zeros(222), np.ones(222)))) # TODO

In [ ]:
embeddings_tsne = TSNE(n_jobs=16).fit_transform(both_embeddings)

point_of_interest = [46,27] # what are these?

def find_idx(e):
    return np.linalg.norm(point_of_interest-e)

def get_idx(e):
    return np.argsort(np.linalg.norm(point_of_interest-e, axis=1))

points = 100
test_embeddings = sorted(embeddings_tsne, key=find_idx)
test_points = np.array(test_embeddings)
center = test_points[0:points].mean(axis=0)
width = np.linalg.norm(point_of_interest-test_points[0:points][-1], axis=-1)
test_embedding_indexs = get_idx(np.array(embeddings_tsne))[:points]
equivalent_points = np.array(embeddings_tsne)[test_embedding_indexs]
center = equivalent_points.mean(axis=0)
cir = plt.Circle(center, width, color='r', fill=False)

In [ ]:
vis_x_cb = embeddings_tsne[:, 0]
vis_y_cb = embeddings_tsne[:, 1]

In [ ]:
fig = plt.figure(figsize=(36,24))
ax = fig.add_subplot(111)
ax.scatter(vis_x_cb, vis_y_cb, color=colors, marker='.')
ax.set_aspect('equal', adjustable='datalim')
#ax.add_patch(cir)
plt.show()

In [ ]:
# embedding comes from bertmodel(**fast_tokenizer(ten_k_netty["text"],
 # max_length = 512, truncation=True, padding=True, return_tensors="pt"
 #)).hidden_states[-1].detach().numpy()
# embeddings[:,:1,:].squeeze()
#    both_embeddings = np.concatenate([embeddings_a, embeddings_b])
# both_embeddings.shape should be (444, 512) because 222 examples * 2 * 512 dimensions